In [ ]:
from bottle import Bottle, route, run, template, get, post, debug, static_file, request, redirect, response
import time
import random
import string
import logging
import logging.handlers

log = logging.getLogger('bottle')
log.setLevel('INFO')
h = logging.handlers.TimedRotatingFileHandler(
    'logs/nlog', when='midnight', backupCount=9999)
f = logging.Formatter('%(asctime)s %(levelname)-8s %(message)s')
h.setFormatter(f)
log.addHandler(h)


secretKey = "vcxv&TTFS98v334sd123fsdhkldasdjghd6DFOUFYWE&FY"


from users import users
from statics import messDict

app = Bottle()


@route('/static/<path:path>', name='static')
def static(path):
    return static_file(path, root='./static')


def checkAuth():
    loginName = request.get_cookie("user", secret=secretKey)
    randStr = request.get_cookie("randStr", secret=secretKey)
    log.info(str(loginName) + ' ' + request.method + ' ' +
             request.url + ' ' + request.environ.get('REMOTE_ADDR'))
    if (loginName in users) and \
        (users[loginName].get("randStr", "") == randStr) and \
        (users[loginName]["loggedIn"] == True) and \
        (time.time() - users[loginName]["lastSeen"] < 3600):
        users[loginName]["lastSeen"] = time.time()
        return loginName
    return redirect('/login')


@route('/login')
@route('/login/')
@route('/login', method='POST')
def login():
    loginName = request.forms.getunicode('inputLogin', default=False)
    password = request.forms.getunicode('inputPassword', default=False)
    randStr = ''.join(random.choice(
        string.ascii_uppercase + string.digits) for _ in range(18))
    
    log.info(str(loginName) + ' ' + request.method + ' ' +
             request.url + ' ' + request.environ.get('REMOTE_ADDR'))
    
    if (loginName in users) and users[loginName]["password"] == password:
        response.set_cookie("user", loginName, secret=secretKey)
        response.set_cookie("randStr", randStr, secret=secretKey)
        users[loginName]["loggedIn"] = True
        users[loginName]["randStr"] = randStr
        users[loginName]["lastSeen"] = time.time()
        
        redirect('/index')
        return True
    else:
        return template('login')
    return template('login')


@route('/')
@route('/index')
@route('/index/')
@route('/index/<message>')
def index(message=''):
    loginName = checkAuth()
    return template('index', message=messDict.get(message, ""), loginName=loginName)

@route('/login/')
@route('/login')
def login():
    return template("login")

@route('/register')
@route('/register/')
def regiser():
    return template("register")

@route('/one_sample_ttest')
@route('/one_sample_ttest/')
def one_sample_ttest():
    return template("one_sample_ttest")

# Debug = True - changes in template are visible immidietly
run(host='localhost', port=8012, debug=True)

Bottle v0.12.19 server starting up (using WSGIRefServer())...
Listening on http://localhost:8012/
Hit Ctrl-C to quit.

127.0.0.1 - - [11/Jan/2022 01:27:55] "GET / HTTP/1.1" 303 0
127.0.0.1 - - [11/Jan/2022 01:27:55] "GET /login HTTP/1.1" 200 3036
127.0.0.1 - - [11/Jan/2022 01:28:02] "POST /login HTTP/1.1" 303 0
127.0.0.1 - - [11/Jan/2022 01:28:02] "GET /index HTTP/1.1" 200 4790
127.0.0.1 - - [11/Jan/2022 01:29:23] "GET /login HTTP/1.1" 200 3060
127.0.0.1 - - [11/Jan/2022 01:29:32] "POST /login HTTP/1.1" 303 0
127.0.0.1 - - [11/Jan/2022 01:29:32] "GET /index HTTP/1.1" 200 4790
127.0.0.1 - - [11/Jan/2022 01:29:36] "GET /one_sample_ttest HTTP/1.1" 200 6226
127.0.0.1 - - [11/Jan/2022 01:29:45] "GET /index HTTP/1.1" 200 4790
